# Fire and Weather Timing Open Question Exploration (v1.0)

Casey A Graff

August 11th, 2017

Exploring the relationships in time between fire detections and weather measurements. Also of interest is when to "cut-off" predictions for day T vs T+1.

In [ ]:
REP_DIR = "/home/cagraff/Documents/dev/fire_prediction/"
SRC_DIR = REP_DIR + 'src/'
DATA_DIR = REP_DIR + 'data/'

# Load system-wide packages
import os
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

# Load project packages
os.chdir(SRC_DIR)
from features.loaders import load_cluster_df
plt.rcParams['figure.figsize'] = [15,7]

In [ ]:
# Load data
cluster_df = load_cluster_df(os.path.join(DATA_DIR, 'processed/cluster/clust_feat_df_5.pkl'))
print list(cluster_df)

In [ ]:
from util.daymonth import day2monthday
def get_covar_dict(fire_dict, df1, df2, year, date_range, myvars):
    dayofyear = date_range[0]
    ret_dict = dict()
    for var in myvars:
        if var == "n_det":
            ret_dict["n_det"] = []
        else:
            ret_dict[var + "1"] = []
            ret_dict[var + "2"] = []
    while dayofyear <= date_range[1]:
        month,day = day2monthday(dayofyear, year % 4 == 0)
        df1_row = df1[(df1.year==year) & (df1.month==month) & (df1.day==day)]
        df2_row = df2[(df2.year==year) & (df2.month==month) & (df2.day==day)]
        for var in myvars:
            if var == "n_det":
                ret_dict["n_det"].append(len(fire_dict[(fire_dict.year == year) & (fire_dict.day == day) & 
                                                       (fire_dict.month == month)]))
            else:
                if len(df1_row):
                    ret_dict[var + "1"].append(float(df1_row[var]))
                else:
                    ret_dict[var + "1"].append(np.nan)
                if len(df2_row):
                    ret_dict[var + "2"].append(float(df2_row[var]))
                else:
                    ret_dict[var + "2"].append(np.nan)
        dayofyear += 1
        
    for var in ret_dict.keys():
        ret_dict[var] = np.array(ret_dict[var])
    return ret_dict

# Covar plot
dist = 75
fb_fires = modis[(np.abs(modis.x - fb_x) < dist) & (np.abs(modis.y - fb_y) < dist)]
year = 2015
fire_season = (133,242)
season_days = np.arange(fire_season[0], fire_season[1]+1)
myvars = ['n_det', 'temp', 'humidity', 'wind', 'rain']
units = ['#', 'C', '%', 'mps', 'mm']
covar_dict = get_covar_dict(fb_fires, fb_gfs, fb_station, year, fire_season, myvars)
covar_dict["temp1"] -= 273.15
covar_dict["wind2"] *= 0.44704
covar_dict["rain2"] *= 25.4

fig, axes = plt.subplots(nrows=len(myvars), ncols=1, figsize=(12,10))
for i, var in enumerate(myvars):
    if not i:
        ax = plt.subplot(511 + i)
        ax1 = ax
    else:
        ax = plt.subplot(511 + i, sharex=ax1)

    if var == 'n_det':
        plt.plot(season_days, covar_dict["n_det"], "r+")
    else:
        plt.plot(season_days, covar_dict[var + "1"], "r--", label="GFS data")
        plt.plot(season_days, covar_dict[var + "2"], "b--", label="Station data")
        plt.legend()
    plt.title("%s (%s)" % (var, units[i]))
fig.tight_layout()
plt.savefig("pics/station_vs_gfs_covar_2013.png")
plt.show()

In [ ]:
MIN_DETECTIONS = 200
fire_season = (133,242)
season_days = np.arange(fire_season[0], fire_season[1]+1)

cluster_counts = []
for i in range(int(max(cluster_df.cluster))):
    sel_df = cluster_df[cluster_df.cluster==i]
    cluster_counts.append((i, np.sum(sel_df.n_det)))
cluster_counts.sort(reverse=True, key=lambda x: x[1])
print [c for c in cluster_counts if c[1] > MIN_DETECTIONS]

In [ ]:
def plot_cluster(cluster_id):
    sel_df = cluster_df[cluster_df.cluster==cluster_id]

    fig, axes = plt.subplots(nrows=6, ncols=1, figsize=(12,10))
    plt.suptitle('Cluster #{}'.format(cluster_id))
    plt.tight_layout(pad=4)

    data_types = [('n_det', 'r+', 'Num Det'), ('rain', '', 'Rain'), ('rain_del_2', '', 'Rain2'), ('humidity', '', 'Humidity'),
                  ('wind', '', 'Wind'), ('temp', '', 'Temp (C)')]

    for i, (type_,form,title) in enumerate(data_types):
        axes[i].plot(sel_df.dayofyear, sel_df[type_], form)
        axes[i].set_title(title)
        
    

plot_cluster(254)
plot_cluster(244)
plot_cluster(984)
